###install dependenciess

In [1]:
!pip install -q bitsandbytes accelerate nltk lightning peft datasets Pillow roboflow

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.1/60.1 MB 15.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 828.5/828.5 kB 35.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.9/89.9 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.8/66.8 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.9/49.9 MB 13.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 56.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.2/4.2 MB 82.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 983.2/983.2 kB 22.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 832.4/832.4 kB 29.0 MB/s eta 0:00:00


In [2]:
from transformers import (BitsAndBytesConfig,
                          LlavaNextForConditionalGeneration,
                          AutoProcessor,
                          AutoTokenizer,
                          AutoModelForCausalLM)

from datasets import load_dataset
from roboflow import Roboflow
from typing import Any, Dict, List, Optional
from PIL import Image
from torch.utils.data import DataLoader
from nltk import edit_distance
from peft import LoraConfig, prepare_model_for_kbit_training, get_peft_model
from PIL import Image
from torch.nn.utils.rnn import pad_sequence
from transformers import AutoTokenizer, AutoModelForCausalLM
from PIL import Image

import torch
import json
import lightning as L
import re
import numpy as np
import bitsandbytes as bnb
import os
import glob
import json
import random
import os
import re
import cv2
import numpy as np
import torch

# optional imports for HuggingFace and YAML parsing
try:
    from datasets import load_dataset
except Exception:
    load_dataset = None

try:
    import yaml
except Exception:
    yaml = None



if torch.cuda.is_available():
    print("PyTorch is connected to GPU.")
    print(f"GPU Device Name: {torch.cuda.get_device_name(0)}")
    print(f"Number of GPUs available: {torch.cuda.device_count()}")
    print(f"Current GPU: {torch.cuda.current_device()}")
else:
    print("PyTorch is not connected to GPU.")

PyTorch is connected to GPU.
GPU Device Name: Tesla T4
Number of GPUs available: 1
Current GPU: 0


###dataset

In [4]:
class LlavaDataset:
    """
    PyTorch-style Dataset (duck-typed; not inheriting torch.utils.data.Dataset to avoid
    import requirement here). Supports two dataset formats:

    - HuggingFace ("hf") : same behavior as your original code: `dataset_name_or_path` is a
      HuggingFace dataset id and we read `sample["ground_truth"]` JSON strings.

    - Roboflow/YOLO ("roboflow") : `dataset_name_or_path` should be the path returned by
      Roboflow's `version.download(...)`. Expects folders like:
         <path>/train/images, <path>/train/labels, <path>/valid/images, <path>/valid/labels
      or similar. Label files are standard YOLO txt files:
         <class_idx> <x_center_norm> <y_center_norm> <w_norm> <h_norm>
    """

    def __init__(
        self,
        dataset_name_or_path: str,
        split: str = "train",
        sort_json_key: bool = True,
        dataset_format: str = "hf",  # "hf" or "roboflow"
        class_names: Optional[List[str]] = None,  # optional list of class names for roboflow mode
        take: Optional[int] = None,  # optional: limit number of samples (like your .take(100))
        transform=None,  # optional transform to apply to PIL image
    ):
        self.split = split
        self.sort_json_key = sort_json_key
        self.dataset_format = dataset_format
        self.transform = transform

        self.gt_token_sequences: List[List[str]] = []

        if dataset_format not in ("hf", "roboflow"):
            raise ValueError('dataset_format must be "hf" or "roboflow"')

        if dataset_format == "hf":
            if load_dataset is None:
                raise RuntimeError("datasets (HuggingFace) library is required for hf format")
            # same logic as original: load HF dataset and expect sample["ground_truth"] JSON string
            ds = load_dataset(dataset_name_or_path, split=self.split)
            if take is not None:
                ds = ds.select(range(min(len(ds), take)))
            self.dataset = ds
            self.dataset_length = len(self.dataset)

            for sample in self.dataset:
                ground_truth = json.loads(sample["ground_truth"])
                if "gt_parses" in ground_truth:
                    assert isinstance(ground_truth["gt_parses"], list)
                    gt_jsons = ground_truth["gt_parses"]
                else:
                    assert "gt_parse" in ground_truth and isinstance(ground_truth["gt_parse"], dict)
                    gt_jsons = [ground_truth["gt_parse"]]

                self.gt_token_sequences.append(
                    [
                        self.json2token(gt_json, sort_json_key=self.sort_json_key)
                        for gt_json in gt_jsons
                    ]
                )

        else:  # roboflow / YOLO format
            # Expect a directory structure like <dataset_name_or_path>/<split>/(images|labels)
            base_path = dataset_name_or_path
            # support split name variants
            split_dir_candidates = [split, "valid" if split == "validation" else None, "validation", "train"]
            split_dir = None
            for cand in split_dir_candidates:
                if cand is None:
                    continue
                cand_path = os.path.join(base_path, cand)
                if os.path.isdir(cand_path):
                    split_dir = cand_path
                    break
            # fallback: sometimes Roboflow places files directly under dataset path in "images" folder
            if split_dir is None:
                # check common patterns
                if os.path.isdir(os.path.join(base_path, "images")):
                    split_dir = base_path
                else:
                    # last resort: try base_path itself
                    split_dir = base_path

            # locate images and labels directories
            images_dir = os.path.join(split_dir, "images")
            labels_dir = os.path.join(split_dir, "labels")
            if not os.path.isdir(images_dir):
                # maybe files are in <split_dir>/*.jpg directly
                images_dir = split_dir

            # Try to load class names from data.yaml if available
            if class_names is None:
                yaml_candidates = [
                    os.path.join(base_path, "data.yaml"),
                    os.path.join(base_path, "dataset.yaml"),
                    os.path.join(base_path, "data.yml"),
                ]
                names = None
                for yc in yaml_candidates:
                    if os.path.exists(yc) and yaml is not None:
                        with open(yc, "r") as f:
                            try:
                                y = yaml.safe_load(f)
                                if isinstance(y, dict) and "names" in y:
                                    names = y["names"]
                                    break
                            except Exception:
                                continue
                class_names = names

            # discover image files
            exts = ["*.jpg", "*.jpeg", "*.png", "*.bmp", "*.tiff"]
            image_paths = []
            for ext in exts:
                image_paths.extend(glob.glob(os.path.join(images_dir, ext)))
            image_paths = sorted(image_paths)
            if take is not None:
                image_paths = image_paths[:take]

            # build mapping index -> class name if provided, else fallback to "class_{idx}"
            self.class_names = class_names if isinstance(class_names, (list, tuple)) else None

            self.image_paths = image_paths
            self.labels_dir = labels_dir
            self.dataset_length = len(self.image_paths)

            for img_path in self.image_paths:
                base = os.path.splitext(os.path.basename(img_path))[0]
                label_path = os.path.join(self.labels_dir, base + ".txt")
                boxes = []
                if os.path.exists(label_path):
                    # need image width/height to convert normalized coords
                    with Image.open(img_path) as im:
                        img_w, img_h = im.size

                    with open(label_path, "r") as f:
                        for line in f:
                            line = line.strip()
                            if not line:
                                continue
                            parts = line.split()
                            if len(parts) < 5:
                                # ignore malformed line
                                continue
                            class_idx = int(float(parts[0]))
                            x_center = float(parts[1])
                            y_center = float(parts[2])
                            w_norm = float(parts[3])
                            h_norm = float(parts[4])

                            # compute absolute coordinates (x_min, y_min, x_max, y_max)
                            cx = x_center * img_w
                            cy = y_center * img_h
                            bw = w_norm * img_w
                            bh = h_norm * img_h
                            x_min = max(0.0, cx - bw / 2.0)
                            y_min = max(0.0, cy - bh / 2.0)
                            x_max = min(img_w, cx + bw / 2.0)
                            y_max = min(img_h, cy + bh / 2.0)

                            if self.class_names and 0 <= class_idx < len(self.class_names):
                                class_name = self.class_names[class_idx]
                            else:
                                class_name = f"class_{class_idx}"

                            boxes.append(
                                {
                                    "class_idx": class_idx,
                                    "class_name": class_name,
                                    "bbox": [x_min, y_min, x_max, y_max],
                                    "bbox_norm": [x_center, y_center, w_norm, h_norm],
                                }
                            )
                else:
                    # no label file: empty boxes list
                    boxes = []

                # construct a `gt_parse` dict similar to HF dataset so json2token handles it
                gt_parse = {"boxes": boxes}
                # if you want a text sequence fallback, put it here:
                # gt_parse["text_sequence"] = "no_text"  # optional

                # we store a list for compatibility (the original code had possibly multiple parses)
                token_seq = self.json2token(gt_parse, sort_json_key=self.sort_json_key)
                # keep as list of alternatives (one element)
                self.gt_token_sequences.append([token_seq])

    def json2token(self, obj: Any, sort_json_key: bool = True):
        """
        Convert an ordered JSON-like python object into a token sequence.
        Mirrors your original implementation.
        """
        if type(obj) == dict:
            if len(obj) == 1 and "text_sequence" in obj:
                return obj["text_sequence"]
            else:
                output = ""
                if sort_json_key:
                    keys = sorted(obj.keys(), reverse=True)
                else:
                    keys = obj.keys()
                for k in keys:
                    output += (
                        fr"<s_{k}>"
                        + self.json2token(obj[k], sort_json_key)
                        + fr"</s_{k}>"
                    )
                return output
        elif type(obj) == list:
            return r"<sep/>".join(
                [self.json2token(item, sort_json_key) for item in obj]
            )
        else:
            obj = str(obj)
            return obj

    def __len__(self) -> int:
        return self.dataset_length

    def __getitem__(self, idx: int):
        """
        Returns:
            image (PIL.Image) , target_sequence (str)
        """
        if self.dataset_format == "hf":
            sample = self.dataset[idx]
            image = sample["image"]
            target_sequence = random.choice(self.gt_token_sequences[idx])
            if self.transform is not None:
                image = self.transform(image)
            return image, target_sequence
        else:
            img_path = self.image_paths[idx]
            image = Image.open(img_path).convert("RGB")
            if self.transform is not None:
                image = self.transform(image)
            target_sequence = random.choice(self.gt_token_sequences[idx])
            return image, target_sequence


In [5]:
rf = Roboflow(api_key="Qu018ajUkOv8KH7Zpl7O")
project = rf.workspace("sejong-x037c").project("fire-smoke-f1fv1")
version = project.version(1)
dataset = version.download("yolov11")
dataset_path = '/content/fire-smoke-1'
train_dataset = LlavaDataset(dataset_path, split="train", dataset_format="roboflow", sort_json_key=False)
val_dataset   = LlavaDataset(dataset_path, split="valid",  dataset_format="roboflow", sort_json_key=False)

loading Roboflow workspace...
loading Roboflow project...



Extracting Dataset Version Zip to fire-smoke-1 in yolov11:: 100%|██████████| 17890/17890 [00:05<00:00, 3164.82it/s]


In [6]:
output_dir = "visualized_train"
os.makedirs(output_dir, exist_ok=True)

for idx in range(min(10, len(val_dataset))):  # visualize first 10 samples
    image, token_seq = val_dataset[idx]
    print(token_seq)
    # Convert PIL -> OpenCV BGR
    image_cv = cv2.cvtColor(np.array(image), cv2.COLOR_RGB2BGR)

    # Extract class name and bbox using regex
    class_names = re.findall(r"<s_class_name>(.*?)</s_class_name>", token_seq)
    bbox_raw = re.findall(r"<s_bbox>(.*?)</s_bbox>", token_seq)

    if not bbox_raw:
        print(f"[{idx}] ⚠️ No bounding boxes found.")
        continue

    for cls_name, bbox_text in zip(class_names, bbox_raw):
        # Split by <sep/> and convert to float
        parts = bbox_text.split("<sep/>")
        if len(parts) != 4:
            continue

        x1, y1, x2, y2 = [float(p.strip()) for p in parts]

        # Convert to integers for OpenCV
        x1, y1, x2, y2 = map(int, [x1, y1, x2, y2])

        # Pick color depending on class name
        color = (0, 255, 0)  # green default
        if "fire" in cls_name.lower():
            color = (0, 0, 255)  # red
        elif "smoke" in cls_name.lower():
            color = (255, 165, 0)  # orange

        # Draw rectangle + label
        cv2.rectangle(image_cv, (x1, y1), (x2, y2), color, 2)
        cv2.putText(
            image_cv,
            cls_name,
            (x1, max(20, y1 - 10)),
            cv2.FONT_HERSHEY_SIMPLEX,
            0.7,
            color,
            2,
            cv2.LINE_AA,
        )

    # Show (optional)
    # cv2.imshow("Annotated", image_cv)
    # cv2.waitKey(0)

    # Save the annotated image
    save_path = os.path.join(output_dir, f"train_{idx}.jpg")
    cv2.imwrite(save_path, image_cv)
    print(f"✅ Saved annotated image: {save_path}")

# cv2.destroyAllWindows()


<s_boxes><s_class_idx>0</s_class_idx><s_class_name>fire</s_class_name><s_bbox>209.5<sep/>353.0<sep/>275.5<sep/>428.0</s_bbox><s_bbox_norm>0.37890625<sep/>0.61015625<sep/>0.103125<sep/>0.1171875</s_bbox_norm><sep/><s_class_idx>0</s_class_idx><s_class_name>fire</s_class_name><s_bbox>289.75<sep/>173.5<sep/>638.25<sep/>412.5</s_bbox><s_bbox_norm>0.725<sep/>0.4578125<sep/>0.54453125<sep/>0.3734375</s_bbox_norm><sep/><s_class_idx>2</s_class_idx><s_class_name>smoke</s_class_name><s_bbox>243.5<sep/>0.25<sep/>635.5<sep/>170.75</s_bbox><s_bbox_norm>0.68671875<sep/>0.13359375<sep/>0.6125<sep/>0.26640625</s_bbox_norm></s_boxes>
✅ Saved annotated image: visualized_train/train_0.jpg
<s_boxes><s_class_idx>0</s_class_idx><s_class_name>fire</s_class_name><s_bbox>267.5<sep/>129.25<sep/>437.5<sep/>426.75</s_bbox><s_bbox_norm>0.55078125<sep/>0.434375<sep/>0.265625<sep/>0.46484375</s_bbox_norm><sep/><s_class_idx>2</s_class_idx><s_class_name>smoke</s_class_name><s_bbox>386.25<sep/>19.0<sep/>506.75<sep/>283.

###Loading Model

In [7]:
MODEL_ID = "apple/FastVLM-0.5B"
IMAGE_TOKEN_INDEX = -200  # what the model code looks for

In [8]:
tok = AutoTokenizer.from_pretrained(MODEL_ID, trust_remote_code=True)


The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.


tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

added_tokens.json:   0%|          | 0.00/80.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/367 [00:00<?, ?B/s]

In [9]:
bnb_config = BitsAndBytesConfig(
            load_in_4bit=True, bnb_4bit_quant_type="nf4", bnb_4bit_compute_dtype=torch.float16
        )
model = AutoModelForCausalLM.from_pretrained(
    MODEL_ID,
    torch_dtype=torch.float32,
    device_map="auto",
    trust_remote_code=True,
    quantization_config=bnb_config
)


config.json: 0.00B [00:00, ?B/s]

llava_qwen.py: 0.00B [00:00, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/apple/FastVLM-0.5B:
- llava_qwen.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.
`torch_dtype` is deprecated! Use `dtype` instead!


model.safetensors:   0%|          | 0.00/1.52G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/100 [00:00<?, ?B/s]

In [10]:
def find_lora_target_modules(model):
    """
    Collects the module names where LoRA adapters should be injected
    (e.g., attention and MLP projections).
    """
    target_keywords = ["q_proj", "k_proj", "v_proj", "o_proj",
                       "gate_proj", "up_proj", "down_proj"]

    target_modules = set()
    for name, module in model.named_modules():
        if any(kw in name for kw in target_keywords):
            # only add the final submodule name (not the full path)
            target_modules.add(name.split(".")[-1])

    return list(target_modules)

lora_config = LoraConfig(
    r=8,
    lora_alpha=8,
    lora_dropout=0.1,
    target_modules=find_lora_target_modules(model),
    init_lora_weights="gaussian",
)

model = prepare_model_for_kbit_training(model)
model = get_peft_model(model, lora_config)

In [11]:
MAX_LENGTH = 128
IMAGE_TOKEN_INDEX = -200

# Load image for single example
# image_path = "test.jpg"
# img = Image.open(image_path).convert("RGB")

# Load tokenizer and model
model_name = "apple/FastVLM-0.5B"
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
# model = AutoModelForCausalLM.from_pretrained(
#     model_name,
#     torch_dtype=torch.float16 if torch.cuda.is_available() else torch.float32,
#     device_map="auto",
#     trust_remote_code=True,
# )

# Prepare image in same dtype/device as model
image_processor = model.get_vision_tower().image_processor
# px = image_processor(images=img, return_tensors="pt")["pixel_values"]
# px = px.to(model.device, dtype=model.dtype)  # convert to float16 if model is float16
# pixel_values_list = [px]

# Chat message for single example
messages = [
    {"role": "user", "content": "<image>\nDescribe this image in detail."}
]

# Tokenize text
full_rendered = tokenizer.apply_chat_template(messages, add_generation_prompt=False, tokenize=False)
pre, post = full_rendered.split("<image>", 1)

pre_ids = tokenizer(pre, return_tensors="pt", add_special_tokens=False).input_ids
post_ids = tokenizer(post, return_tensors="pt", add_special_tokens=False).input_ids
img_tok = torch.tensor([[IMAGE_TOKEN_INDEX]], dtype=pre_ids.dtype)

# Build input_ids
input_ids = torch.cat([pre_ids, img_tok, post_ids], dim=1).to(model.device)  # stays as int64
attention_mask = torch.ones_like(input_ids, device=model.device)  # stays as int64



def train_collate_fn(examples):
    """
    Prepares a batch of training data for a model that processes images and text.

    Args:
        examples (list): A list of tuples where each tuple contains an image and the corresponding ground truth.

    Returns:
        tuple: A tuple containing:
            - input_ids (torch.Tensor): Tokenized input IDs of the text prompts.
            - attention_mask (torch.Tensor): Attention mask for the input IDs.
            - pixel_values (torch.Tensor): Preprocessed pixel values of the images.
            - image_sizes (list): List of original sizes of the images.
            - labels (torch.Tensor): Labels for training, with prompt tokens and padding replaced by -100.
    """
    input_ids_list = []
    attention_mask_list = []
    pixel_values_list = []
    image_sizes = []
    labels_list = []

    tok = tokenizer
    # Assuming image_processor is defined globally or passed; here it's global

    for example in examples:
        image, ground_truth = example
        image_sizes.append(image.size)

        px = image_processor(image, return_tensors="pt")["pixel_values"][0]
        pixel_values_list.append(px)

        messages = [
            {"role": "user", "content": "<image>\nExtract JSON"},
            {"role": "assistant", "content": ground_truth}
        ]

        full_rendered = tok.apply_chat_template(messages, add_generation_prompt=False, tokenize=False)
        pre, post = full_rendered.split("<image>", 1)
        pre_ids = tok(pre, add_special_tokens=False, return_tensors="pt", truncation=True, max_length=MAX_LENGTH).input_ids[0]
        post_ids = tok(post, add_special_tokens=False, return_tensors="pt", truncation=True, max_length=MAX_LENGTH).input_ids[0]
        img_tok = torch.tensor([IMAGE_TOKEN_INDEX], dtype=pre_ids.dtype)
        input_ids = torch.cat([pre_ids, img_tok, post_ids])
        attention_mask = torch.ones_like(input_ids)

        prompt_messages = messages[:-1]
        prompt_rendered = tok.apply_chat_template(prompt_messages, add_generation_prompt=True, tokenize=False)
        pre_p, post_p = prompt_rendered.split("<image>", 1)
        pre_ids_p = tok(pre_p, add_special_tokens=False, return_tensors="pt").input_ids[0]
        post_ids_p = tok(post_p, add_special_tokens=False, return_tensors="pt").input_ids[0]
        img_tok_p = torch.tensor([IMAGE_TOKEN_INDEX], dtype=pre_ids.dtype)
        prompt_ids = torch.cat([pre_ids_p, img_tok_p, post_ids_p])
        prompt_len = len(prompt_ids)

        labels = input_ids.clone()
        if prompt_len < len(input_ids):
            labels[:prompt_len] = -100

        input_ids_list.append(input_ids)
        attention_mask_list.append(attention_mask)
        labels_list.append(labels)

    input_ids = pad_sequence(input_ids_list, batch_first=True, padding_value=tok.pad_token_id)
    attention_mask = pad_sequence(attention_mask_list, batch_first=True, padding_value=0)
    labels = pad_sequence(labels_list, batch_first=True, padding_value=-100)
    pixel_values = torch.stack(pixel_values_list)

    return input_ids, attention_mask, pixel_values, image_sizes, labels


def eval_collate_fn(examples):
    """
    Prepares a batch of evaluation data for a model that processes images and text.

    Args:
        examples (list): A list of tuples where each tuple contains an image and the corresponding ground truth.

    Returns:
        tuple: A tuple containing:
            - input_ids (torch.Tensor): Tokenized input IDs of the text prompts.
            - attention_mask (torch.Tensor): Attention mask for the input IDs.
            - pixel_values (torch.Tensor): Preprocessed pixel values of the images.
            - image_sizes (list): List of original sizes of the images.
            - answers (list): List of ground truth answers for evaluation.
    """
    input_ids_list = []
    attention_mask_list = []
    pixel_values_list = []
    image_sizes = []
    answers = []

    tok = tokenizer
    # Assuming image_processor is defined globally or passed; here it's global

    for example in examples:
        image, ground_truth = example
        image_sizes.append(image.size)

        px = image_processor(image, return_tensors="pt")["pixel_values"][0]
        pixel_values_list.append(px)

        messages = [
            {"role": "user", "content": "<image>\nExtract JSON"}
        ]

        rendered = tok.apply_chat_template(messages, add_generation_prompt=True, tokenize=False)
        pre, post = rendered.split("<image>", 1)
        pre_ids = tok(pre, add_special_tokens=False, return_tensors="pt", truncation=True, max_length=MAX_LENGTH).input_ids[0]
        post_ids = tok(post, add_special_tokens=False, return_tensors="pt", truncation=True, max_length=MAX_LENGTH).input_ids[0]
        img_tok = torch.tensor([IMAGE_TOKEN_INDEX], dtype=pre_ids.dtype)
        input_ids = torch.cat([pre_ids, img_tok, post_ids])
        attention_mask = torch.ones_like(input_ids)

        input_ids_list.append(input_ids)
        attention_mask_list.append(attention_mask)
        answers.append(ground_truth)

    input_ids = pad_sequence(input_ids_list, batch_first=True, padding_value=tok.pad_token_id)
    attention_mask = pad_sequence(attention_mask_list, batch_first=True, padding_value=0)
    pixel_values = torch.stack(pixel_values_list)

    return input_ids, attention_mask, pixel_values, image_sizes, answers

In [12]:
class LlavaModelPLModule(L.LightningModule):
    """
    A PyTorch Lightning module for training and validating a multimodal model that processes images and text.

    Attributes:
        config (dict): Configuration dictionary containing model hyperparameters and settings.
        processor (object): A processor object for handling text and image pre-processing.
        model (torch.nn.Module): The model to be trained and evaluated.

    Methods:
        training_step(batch, batch_idx):
            Executes a single training step, computing the loss and logging it.

        validation_step(batch, batch_idx, dataset_idx=0):
            Executes a single validation step, generating predictions, comparing them to ground truth, and logging the normalized edit distance.

        configure_optimizers():
            Sets up the optimizer and optionally, learning rate scheduler for the training process.

        train_dataloader():
            Returns a DataLoader for the training dataset.

        val_dataloader():
            Returns a DataLoader for the validation dataset.
    """
    def __init__(self, config, processor, model):
        super().__init__()
        self.config = config
        self.processor = processor
        self.model = model

        self.batch_size = config.get("batch_size")

    def training_step(self, batch, batch_idx):
        """
        Performs a single step of training.

        Args:
            batch (tuple): A tuple containing input_ids, attention_mask, pixel_values, image_sizes, and labels.
            batch_idx (int): The index of the current batch.

        Returns:
            torch.Tensor: The computed loss for the batch.
        """

        input_ids, attention_mask, pixel_values, image_sizes, labels = batch

        outputs = self.model(input_ids=input_ids,
                            attention_mask=attention_mask,
                            images=pixel_values,
                            image_sizes=image_sizes,
                            labels=labels
                          )
        loss = outputs.loss
        print(loss)
        self.log("train_loss", loss)

        return loss

    def validation_step(self, batch, batch_idx, dataset_idx=0):
        """
        Performs a single step of validation, generating predictions and computing the normalized edit distance.

        Args:
            batch (tuple): A tuple containing input_ids, attention_mask, pixel_values, image_sizes, and answers.
            batch_idx (int): The index of the current batch.
            dataset_idx (int, optional): Index of the dataset in case of multiple datasets. Defaults to 0.

        Returns:
            list: A list of normalized edit distances between predictions and ground truth answers.
        """

        input_ids, attention_mask, pixel_values, image_sizes, answers = batch
        # print(batch)
        # autoregressively generate token IDs
        # generated_ids = self.model(input_ids=input_ids, attention_mask=attention_mask,
        #                                images=pixel_values, image_sizes=image_sizes)
        # print(generated_ids)
        print('----',batch,'----')

        out = model.generate(
            inputs=input_ids,
            attention_mask=attention_mask,
            images=pixel_values,
            max_new_tokens=128,
        )
        print(tok.decode(out[0], skip_special_tokens=True))
        # # turn them back into text, chopping of the prompt
        # # important: we don't skip special tokens here, because we want to see them in the output
        # predictions = self.processor.batch_decode(generated_ids[:, input_ids.size(1):], skip_special_tokens=True)
        # scores = []
        # for pred, answer in zip(predictions, answers):
        #     pred = re.sub(r"(?:(?<=>) | (?=</s_))", "", pred)
        #     scores.append(edit_distance(pred, answer) / max(len(pred), len(answer)))

        #     if self.config.get("verbose", False) and len(scores) == 1:
        #         print(f"Prediction: {pred}")
        #         print(f"    Answer: {answer}")
        #         print(f" Normed ED: {scores[0]}")

        # self.log("val_edit_distance", np.mean(scores))

        return None

    def configure_optimizers(self):
        """
        Configures the optimizer for training.

        Returns:
            torch.optim.Optimizer: The optimizer for training.
        """
        # you could also add a learning rate scheduler if you want
        optimizer = torch.optim.AdamW(self.parameters(), lr=self.config.get("lr"))

        return optimizer

    def train_dataloader(self):
        """
        Returns the DataLoader for the training dataset.

        Returns:
            DataLoader: The DataLoader for the training dataset.
        """
        return DataLoader(train_dataset, collate_fn=train_collate_fn, batch_size=self.batch_size, shuffle=True, num_workers=4)

    def val_dataloader(self):
        """
        Returns the DataLoader for the validation dataset.

        Returns:
            DataLoader: The DataLoader for the validation dataset.
        """
        return DataLoader(val_dataset, collate_fn=eval_collate_fn, batch_size=self.batch_size, shuffle=False, num_workers=4)

In [13]:
config = {"max_epochs": 10,
          "check_val_every_n_epoch": 1,
          "gradient_clip_val": 1.0,
          "accumulate_grad_batches": 8,
          "lr": 1e-4,
          "batch_size": 1,
          # "seed":2022,
          "num_nodes": 1,
          "warmup_steps": 50,
          "result_path": "./result",
          "verbose": True,
          "num_workers": 4
}

model_module = LlavaModelPLModule(config, tokenizer, model)

In [ ]:
trainer = L.Trainer(
        accelerator="gpu",
        devices=[0],
        max_epochs=config.get("max_epochs"),
        accumulate_grad_batches=config.get("accumulate_grad_batches"),
        check_val_every_n_epoch=config.get("check_val_every_n_epoch"),
        gradient_clip_val=config.get("gradient_clip_val"),
        precision="16",
        limit_val_batches=5,
        num_sanity_val_steps=0,
        # logger=wandb_logger,
)

trainer.fit(model_module)

/usr/local/lib/python3.12/dist-packages/lightning/fabric/connector.py:571: `precision=16` is supported for historical reasons but its usage is discouraged. Please set your precision to 16-mixed instead!
INFO: Using 16bit Automatic Mixed Precision (AMP)
INFO:lightning.pytorch.utilities.rank_zero:Using 16bit Automatic Mixed Precision (AMP)
INFO: 💡 Tip: For seamless cloud uploads and versioning, try installing [litmodels](https://pypi.org/project/litmodels/) to enable LitModelCheckpoint, which syncs automatically with the Lightning model registry.
INFO:lightning.pytorch.utilities.rank_zero:💡 Tip: For seamless cloud uploads and versioning, try installing [litmodels](https://pypi.org/project/litmodels/) to enable LitModelCheckpoint, which syncs automatically with the Lightning model registry.
INFO: GPU available: True (cuda), used: True
INFO:lightning.pytorch.utilities.rank_zero:GPU available: True (cuda), used: True
INFO: TPU available: False, using: 0 TPU cores
INFO:lightning.pytorch.util

Training: |          | 0/? [00:00<?, ?it/s]

Streaming output truncated to the last 5000 lines.
tensor(0.4338, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.4400, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.4160, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.4279, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.4526, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.4176, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.4226, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.4172, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.4271, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.4347, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.4395, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.3873, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.3825, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.4353, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.5049, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.3704, device='cuda:0', grad_fn=<NllLossB

Validation: |          | 0/? [00:00<?, ?it/s]

Streaming output truncated to the last 5000 lines.
tensor(0.3150, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.3171, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.3272, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.2725, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.3529, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.3572, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.3007, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.3501, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.3689, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.3175, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.3392, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.3412, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.2915, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.3007, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.2889, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.3194, device='cuda:0', grad_fn=<NllLossB

Validation: |          | 0/? [00:00<?, ?it/s]

---- (tensor([[151644,   8948,    198,   2610,    525,    264,  10950,  17847,     13,
         151645,    198, 151644,    872,    198,   -200,    198,  28959,   4718,
         151645,    198, 151644,  77091,    198]], device='cuda:0'), tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]],
       device='cuda:0'), tensor([[[[0.7922, 0.7922, 0.7922,  ..., 0.5412, 0.5373, 0.5373],
          [0.7922, 0.7922, 0.7922,  ..., 0.5412, 0.5373, 0.5373],
          [0.7922, 0.7922, 0.7922,  ..., 0.5412, 0.5373, 0.5373],
          ...,
          [0.4706, 0.4706, 0.4706,  ..., 0.5765, 0.5804, 0.5804],
          [0.4745, 0.4745, 0.4784,  ..., 0.5804, 0.5843, 0.5843],
          [0.4784, 0.4784, 0.4824,  ..., 0.5843, 0.5882, 0.5882]],

         [[0.8353, 0.8353, 0.8353,  ..., 0.5373, 0.5333, 0.5333],
          [0.8353, 0.8353, 0.8353,  ..., 0.5373, 0.5333, 0.5333],
          [0.8353, 0.8353, 0.8353,  ..., 0.5373, 0.5333, 0.5333],
          ...,
          [0.4392, 0.4392, 0.439